In [4]:
import caffe
import numpy as np
import matplotlib.pyplot as plt
import cPickle

In [5]:
def get_net_output(inputs, net, blob_names):
    '''
    This function is used to show the output for a specific layer
    Inputs:
        inputs: a dictionary e.g. {"data": xxx, "label": xxx}
            the value for the key "data" should be in the form of
            a list of np array.  
        net: an instance of caffe.Net
        blob_names: names of blobs you want. Each blob name have to 
        
    Outputs:
        
    '''
    input_shape = list(net.blobs['data'].shape)
    input_shape[0] = len(inputs)
    net.blobs['data'].reshape(*tuple(input_shape))

    r_list = [{} for i in range(len(inputs))]
    
    for i_ind in range(len(inputs)):
        net.blobs['data'].data[i_ind, ...] = inputs[i_ind]
    net.forward()
    
    for blob_name in blob_names:
        data_np = net.blobs[blob_name].data
        for i_ind in range(len(inputs)):
            r_list[i_ind][blob_name] = np.copy(data_np[i_ind])
    return r_list

In [6]:
if __name__ == "__main__":
    # the variable for either using a GPU (set to 1) to compute 
    # or a CPU (set to 0)    
    GPU = 1;
    proto_root = "/mnt/disk1/bugfree/PAT/pat1m-sgd-03-03-2016-0.01"
    # NOTE: this prototext have to be a deploy version of the prototxt
    # It is not the one, we used for training
    # net_deploy = proto_root + "/net.deploy"
    net_deploy = "/mnt/disk1/bugfree/PAT/pat1m-sgd0.01" + "/net.deploy"
    model = proto_root + "/_iter_20000000.caffemodel"
    data_root = "/home/bugfree/Workspace/caffe/python_ext/data/pat"
    fdata = data_root + "/pat_1w_test.cpickle"
    data_layer_name = 'Python1'
    
    # set the GPU/CPU mode
    if GPU == 1:
        caffe.set_device(0)
        caffe.set_mode_gpu()
    else:
        caffe.set_mode_cpu()
             
    # load the net from net prototext
    # in the same time load the caffemodel (it contains the model parameters)
    net = caffe.Net(net_deploy,model,caffe.TEST)
    
    # load data 
    [data,label] = cPickle.load(open(fdata,'rb')) 
    print data.shape
    
    # configure preprocessing
    input_shape = list(net.blobs[data_layer_name].data.shape)
    # print input_shape
    # reshape the data layer on the fly
    net.blobs[data_layer_name].reshape(data.shape[0],data.shape[1])
    # load the data to the caffe net
    net.blobs[data_layer_name].data[...] = data
    # compute 
    out = net.forward()
    prediction = out['InnerProduct4']
    
    # for i in range(prediction.shape[0]):
    for i in range(5):
        ground = label[i,:]
        pred = prediction[i,:]
        print ground
        print pred
        print "======================="
        
    # check the performance for each dimension
    print sum((label - prediction)**2,0)/prediction.shape[0]
    print sum(sum((label - prediction)**2,0)/prediction.shape[0])
    
    
    
    # print the blob's name and the shape of data
    # print [(k, v.data.shape) for k, v in net.blobs.items()]
    # print the weight and bias terms for each layers that contains parameter
    # print [(k, v[0].data.shape, v[1].data.shape) for k, v in net.params.items()]
    
    
    

(10000, 300)
[ 0.52132648 -0.85911542  1.15715003 -0.2224991  -0.36600831  2.88458896
 -1.278391    2.65847206  1.895015   -0.3870267  -0.04740342  1.26391399
 -0.25033951 -0.60250711 -0.91032451  1.06851101  0.0405428   0.81675321
  0.87555909 -0.56480789  0.296653   -2.04865694 -0.867199   -0.1689807
  0.06679945 -0.67735189 -2.06812     1.11081398 -0.01826483 -0.57900101
  1.16692698 -1.98820901 -0.17207409]
[ 1.06486464 -1.08471298  1.04569256 -0.13129896  0.91478741  0.46107996
  0.67855179  0.81404829  0.81376314  0.31046534  0.31160545  0.54732704
  0.35257691 -0.01455557 -0.20382285 -0.10423303  0.22592348  0.61598301
  0.50398993 -0.26422769 -1.13804162 -1.03238928 -0.02399355  0.46598691
 -0.17483974 -0.94266921 -0.71167761  0.24311832  0.12155968 -0.77183849
  1.84511757 -0.26117742 -0.03221133]
[ 0.1703514   0.34684679  0.06819048 -0.92202562  1.31649804  1.397205
 -0.80215061 -0.43620881 -0.46481779  1.02041602  0.1341411  -2.73790503
  2.7496531   0.78589731  0.82257962 -